In [11]:
import pandas as pd
import sys
sys.path.append('../..')
from scripts import util

In [12]:
conn = util.connect_rds()

In [13]:
q = 'select * from rws_schema.ongevallen;'

In [14]:
df = pd.read_sql(q, con=conn)

In [15]:
df.shape

(176112, 14)

In [17]:
knmi = pd.read_sql('select * from rws_schema.knmi_data;', con=conn)

In [46]:
knmi.loc[:,'YYYYMMDD'] = pd.to_datetime(knmi.YYYYMMDD)

In [20]:
knmi.head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
0,391.0,2012-01-01,1.0,220.0,40.0,40.0,80.0,107.0,NaN,103.0,...,NaN,NaN,97.0,NaN,6.0,None,None,None,None,None
1,391.0,2012-01-01,2.0,220.0,40.0,40.0,70.0,106.0,NaN,106.0,...,NaN,NaN,100.0,NaN,6.0,None,None,None,None,None
2,391.0,2012-01-01,3.0,210.0,40.0,40.0,70.0,107.0,NaN,106.0,...,NaN,NaN,99.0,NaN,6.0,None,None,None,None,None
3,391.0,2012-01-01,4.0,210.0,40.0,40.0,70.0,108.0,NaN,107.0,...,NaN,NaN,99.0,NaN,6.0,None,None,None,None,None
4,391.0,2012-01-01,5.0,220.0,50.0,50.0,90.0,110.0,NaN,108.0,...,NaN,NaN,99.0,NaN,6.0,None,None,None,None,None


In [32]:
t = pd.to_datetime(knmi.YYYYMMDD)

In [39]:
df.head()

,id,source,datetime,direction,x,y,ending,weg,key,hecto,hectomtrng,dvk_letter,pos_tv_wol,hecto_key
0,90088.0,NDW,2018-02-19 14:56:30,None,6.759355,52.241428,None,35.0,35.608.Li,60.8,608.0,None,Li,497347012.2017-11-01.608
1,90087.0,NDW,2018-02-19 14:25:46,None,5.147580,51.941662,None,2.0,2.795.Li,79.5,795.0,None,Li,275279004.2017-09-01.795
2,90086.0,NDW,2018-02-16 06:31:28,None,4.423963,52.051788,None,12.0,12.99.Re,9.9,99.0,None,Re,177303013.2017-12-01.99
3,90085.0,NDW,2018-02-15 15:11:22,None,4.564720,51.884312,None,16.0,16.236.y,23.6,236.0,y,Li,196267066.2014-11-01.236
4,90084.0,NDW,2018-02-15 10:51:55,None,4.700294,52.384201,None,200.0,200.97.c,9.7,97.0,c,Li,216377018.2015-04-01.97


In [43]:
df.head()t

,id,source,datetime,direction,x,y,ending,weg,key,hecto,hectomtrng,dvk_letter,pos_tv_wol,hecto_key
0,90088.0,NDW,2018-02-19 14:56:30,None,6.759355,52.241428,None,35.0,35.608.Li,60.8,608.0,None,Li,497347012.2017-11-01.608
1,90087.0,NDW,2018-02-19 14:25:46,None,5.147580,51.941662,None,2.0,2.795.Li,79.5,795.0,None,Li,275279004.2017-09-01.795
2,90086.0,NDW,2018-02-16 06:31:28,None,4.423963,52.051788,None,12.0,12.99.Re,9.9,99.0,None,Re,177303013.2017-12-01.99
3,90085.0,NDW,2018-02-15 15:11:22,None,4.564720,51.884312,None,16.0,16.236.y,23.6,236.0,y,Li,196267066.2014-11-01.236
4,90084.0,NDW,2018-02-15 10:51:55,None,4.700294,52.384201,None,200.0,200.97.c,9.7,97.0,c,Li,216377018.2015-04-01.97


In [44]:
trial_time = df.datetime[0]

In [45]:
trial_time

Timestamp('2018-02-19 14:56:30')

In [76]:
get_weather(trial_time, 391, 'DD', knmi_data=knmi)

160.0

0    2012-01-01
1    2012-01-01
2    2012-01-01
3    2012-01-01
4    2012-01-01
Name: YYYYMMDD, dtype: object

In [70]:
knmi.YYYYMMDD.dtype == '<M8[ns]'

True

In [72]:
from warnings import 

warnings.WarningMessage

In [75]:
def get_weather(timestamp, station, weather_type, knmi_data):
    '''
    timestamp datetime
    timestamp of the accident - will be used to find the nearest weather timestamp
    
    station, float
    the weather station code for which the weather is required. This would be the closed weather
    station to the accident point
    
    weather_type string
    Eg 'DD' - the value of precipitation that's required
    These have to be codes as in the dataframe.
    
    knmi_data DataFrame
    KNMI dataframe
    
    return one value - whichever type the column requested is
    
    #TODO!: We are getting weather from the last hour, so even if it's 11:59 its get weather reported
    at 11 and not 12.
    
    '''
    
    
    
    if knmi_data.YYYYMMDD.dtype!='<M8[ns]':
        raise ValueError("Please convert YYYYMMDD column into datetime using pd.to_datetime first")
    
    date = pd.to_datetime(timestamp.date())
    hour = timestamp.hour
    
    
    t = knmi_data.loc[((knmi_data.YYYYMMDD==date)&(knmi_data.HH==hour)&(knmi_data.STN==station)), weather_type].values[0]
    
    return t
    
    
    

In [77]:
from scripts import util

<module 'scripts.util' from '../../scripts/util.py'>